# 뉴스 & 블로그 - 데이터 전처리 및 통합 감성 분석

1. 프로젝트 개요

'카카오톡 업데이트 사용자 피드백 분석' 프로젝트에서 두번째 단계로 수집된 뉴스 및 블로그의 원본 데이터를 <b>분석에 즉시 활용 가능한 '정제된 감성 데이터셋'</b>으로 변환하는 전체 과정을 담고 있습니다.

2. 주요 프로세스 및 핵심 전략

다양한 형태와 품질의 원본 데이터를 일관성 있는 고품질 데이터로 변환하기 위해, 다음과 같은 다단계 정제 및 분석 파이프라인을 구축했습니다.

- 광고성 콘텐츠 제거:
분석의 정확도를 저해하는 광고성/스팸성 게시물(특히 블로그)을 제거하기 위해, 사전에 정의된 '제외 키워드' 목록을 바탕으로 '제목'을 필터링하여 노이즈를 1차적으로 제거했습니다.

- 비정형 날짜 데이터 표준화 (Robust Date Parsing):
'2025. 11. 18.', '오후 1:53:26', 'Nov 18, 2025' 등, 수집된 데이터에 혼재된 수십 가지의 비정형 날짜 텍스트를 YYYY-MM-DD HH:MM:SS 형식으로 통일하는 <b>'강력한(Robust) 날짜 표준화 함수'</b>를 직접 구현했습니다.
이 함수는 정규 표현식, 조건문 등을 복합적으로 사용하여 다양한 예외 케이스에 대응함으로써, 날짜 변환 실패율(NaT)을 최소화하고 정확한 시계열 분석의 기반을 마련했습니다.

- 다차원 NLP 감성 분석:
단순히 '긍정/부정'으로만 분류하는 것을 넘어, 한국어 리뷰 데이터에 특화된 NLP 모델(matthewburke/korean_sentiment)을 사용하여 '제목'과 '본문' 각각에 대한 '긍정', '부정', '중립'의 '확률 점수(Score)'를 모두 추출했습니다.
이를 통해, 단순히 "이 기사는 긍정이다"가 아니라, "이 기사의 제목은 긍정 점수가 80%이지만, 본문은 부정 점수가 60%이다" 와 같이, 콘텐츠의 다층적인 감성을 정량적으로 분석할 수 있는 기반을 구축했습니다.

- 최종 시각화 및 검증:
모든 정제 및 분석이 완료된 후, '날짜별 제목/본문 긍/부정 기사 수' 그래프를 시각화하여, 데이터의 전체적인 흐름과 패턴이 우리의 상식과 일치하는지 최종적으로 검증하는 단계를 거쳤습니다.

3. 결과물

모든 뉴스/블로그 기사에 표준화된 날짜 정보와 다차원 감성 점수가 부여된 최종 분석용 데이터셋(final_..._with_sentiment.pkl)이 생성되었습니다.

In [ ]:
# 그래프 한글 나눔 폰트 설치 및 설정
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

# 런타임 > 세션 다시 시작 실행

In [ ]:
# 필요 라이브러리 설치 및 임포트
!pip install transformers torch sentencepiece

import torch
import pandas as pd
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification
from tqdm.auto import tqdm
tqdm.pandas()
import re
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns

# Matplotlib에서 한글 폰트 설정
plt.rc('font', family='NanumBarunGothic')
plt.rcParams['axes.unicode_minus'] = False # 마이너스 폰트 깨짐 방지

In [ ]:
# 데이터 로드 (블로그와 뉴스의 컬럼명이 달라서 확인 후 수정해서 적용할 것)
try:
    total_df = pd.read_pickle("/content/drive/MyDrive/Colab Notebooks/final_blog_all_articles.pkl")
    print("데이터 파일을 성공적으로 불러왔습니다.")
except FileNotFoundError:
    print("오류: pkl 파일이 없습니다. 파일명을 확인해주세요.")

In [ ]:
# 1. 제목 컬럼에 지정한 텍스트가 포함된 경우 제거 - 전처리 작업
remove_keywords = ['ROHM', '광고', '홍보', '스팸','나는솔로', '헌옷','커튼', '학원', '온라인', '소속사'] # 수집된 블로그 글 중 광고용 텍스트
pattern = '|'.join(remove_keywords)
total_df = total_df[~total_df['제목'].str.contains(pattern, regex=True, case=False)]

# 중간 저장
total_df.to_pickle("/content/drive/MyDrive/Colab Notebooks/final_blog_all_cleaned_articles.pkl")

In [ ]:
# 2. 날짜 오류 수정 - 전처리 작업
# --------------------------------------------------------------------------
# 2-1. 날짜 변환 실패 현황 파악
# --------------------------------------------------------------------------
total_df['datetime_temp'] = pd.to_datetime(total_df['작성일'], errors='coerce')

nat_df = total_df[total_df['datetime_temp'].isnull()]

print(f"총 데이터: {len(total_df)}건")
print(f"날짜 변환 실패(NaT): {len(nat_df)}건")

if not nat_df.empty:
    print("\n--- [ 날짜 변환 실패 상위 18개 샘플 ] ---")
    # 어떤 형식의 텍스트가 실패했는지 눈으로 확인
    display(nat_df[[ '제목', '검색키워드', '작성일']].head(18))

total_df = total_df.drop(columns=['datetime_temp'])

# --------------------------------------------------------------------------
# 2-2. 날짜 정제 함수
# --------------------------------------------------------------------------
def standardize_date(date_string):
    # 다양한 형식의 날짜 텍스트를 'YYYY-MM-DD HH:MM:SS' 표준 형식으로 변환. 시간이 없으면 00:00:00으로 설정
    text = str(date_string).strip()

    try:
        # -----------------------------------------------------------
        # 1순위: ISO 8601 형식 (T와 +가 있는 경우)
        # -----------------------------------------------------------
        if 'T' in text and '+' in text:
            return pd.to_datetime(text).strftime('%Y-%m-%d %H:%M:%S')

        # -----------------------------------------------------------
        # 2순위: '오전/오후'가 포함된 한글 날짜 형식
        # -----------------------------------------------------------
        if '오전' in text or '오후' in text:
            processed_text = text.replace('.', ' ').replace('오전', 'AM').replace('오후', 'PM')
            processed_text = re.sub(r'\s+', ' ', processed_text).strip()
            for fmt in ('%Y-%m-%d %p %I:%M', '%Y %m %d %p %I:%M'):
                try:
                    return pd.to_datetime(processed_text, format=fmt).strftime('%Y-%m-%d %H:%M:%S')
                except ValueError:
                    continue

        # -----------------------------------------------------------
        # 3순위: 정규표현식을 이용한 숫자 추출
        # -----------------------------------------------------------
        # (A) "YYYY.MM.DD HH:MM:SS" 또는 "YYYY.MM.DD" 형태 추출
        # 날짜(연-월-일)는 필수, 시간은 선택(옵션)으로 변경했습니다.
        match_full = re.search(r'(\d{4})[-./\s]+(\d{1,2})[-./\s]+(\d{1,2})', text)

        if match_full:
            year, month, day = map(int, match_full.groups())

            # 시간 정보가 있는지 추가로 확인
            match_time = re.search(r'(\d{1,2}):(\d{2})(:(\d{2}))?', text)

            if match_time:
                # 시간이 있으면 그 시간을 사용
                hour, minute = int(match_time.group(1)), int(match_time.group(2))
                second = int(match_time.group(4)) if match_time.group(4) else 0
            else:
                # 시간이 없으면 00:00:00으로 설정
                hour, minute, second = 0, 0, 0

            return f"{year:04d}-{month:02d}-{day:02d} {hour:02d}:{minute:02d}:{second:02d}"

        # "MM.DD" 형태 (연도 없음) -> "09.18" 같은 케이스 : 연도가 없으면 '올해(시스템 연도)' 또는 특정 연도로 가정
        match_short = re.search(r'^(\d{1,2})[-./](\d{1,2})$', text)
        if match_short:
            month, day = map(int, match_short.groups())
            # 현재 연도를 자동으로 가져옴 (필요시 2025 등으로 고정 가능)
            current_year = datetime.now().year
            return f"{current_year:04d}-{month:02d}-{day:02d} 00:00:00"

        # 위 모든 경우에 해당하지 않으면 원래 텍스트 반환
        return text

    except Exception:
        return text

# --------------------------------------------------------------------------
# 2-3. 일괄 적용 및 최종 검증
# --------------------------------------------------------------------------
print("\n'작성일' 컬럼에 날짜 표준화 함수를 적용합니다...")
total_df['작성일_정제'] = total_df['작성일'].apply(standardize_date)

# 정제된 날짜 컬럼을 datetime 형식으로 최종 변환
total_df['작성일_datetime'] = pd.to_datetime(total_df['작성일_정제'], errors='coerce')

# 여전히 NaT가 남아있는지 최종 확인
final_nat_count = total_df['작성일_datetime'].isnull().sum()
print(f"\n최종 날짜 변환 후 남은 실패(NaT) 건수: {final_nat_count}건")

if final_nat_count > 0:
    print("\n--- [ 여전히 실패하는 샘플 ] ---")
    display(total_df[total_df['작성일_datetime'].isnull()][['제목', '검색키워드', '작성일', '작성일_정제']])

# 결과 확인
print("\n--- [ 날짜 정제 최종 결과 샘플 ] ---")
display(total_df[['제목', '작성일', '작성일_정제','작성일_datetime']].head())

# 데이터프레임을 다시 저장
total_df.to_pickle("/content/drive/MyDrive/Colab Notebooks/final_crawled_blogs_all_cleaned_date.pkl")

In [ ]:
# 3. 감성 분석 모델 적용
# --------------------------------------------------------------------------
# 3-1. 감성 분석 모델 로드
# --------------------------------------------------------------------------
model_name = "matthewburke/korean_sentiment"

sentiment_classifier = pipeline(
    "sentiment-analysis",
    model=model_name,
    return_all_scores=True
)
print("감성 분석 모델 로드 완료! (모든 점수 출력 모드)")

# --------------------------------------------------------------------------
# 3-2. 감성 분석 함수 정의
# --------------------------------------------------------------------------
def analyze_all_sentiments(text):
    # 텍스트에 대해 '긍정', '부정', '중립' 점수를 모두 반환하는 함수

    try:
        if pd.isna(text) or not text.strip():
            return 0.0, 0.0, 0.0 # 순서 : 긍정, 부정, 중립

        truncated_text = str(text)[:512]

        # 감성 분석 실행 -> [{'label': 'LABEL_1', 'score': ...}, {'label': 'LABEL_0', ...}, ...]
        result = sentiment_classifier(truncated_text)[0]

        scores = {item['label']: item['score'] for item in result}

        positive_score = scores.get('LABEL_1', 0.0)
        negative_score = scores.get('LABEL_0', 0.0)
        neutral_score = scores.get('LABEL_2', 0.0)

        return positive_score, negative_score, neutral_score

    except Exception as e:
        print(f"오류 발생: {e}") # 오류 발생 시 로그 출력
        return 0.0, 0.0, 0.0

# --------------------------------------------------------------------------
# 3-3. '제목'과 '본문'에 감성 분석 적용
# --------------------------------------------------------------------------
# 제목 감성 분석
print("\n'제목_정제' 컬럼에 대한 감성 분석을 시작합니다... (모든 점수 추출)")
title_sentiments = total_df['제목_정제'].progress_apply(analyze_all_sentiments)

# 긍정, 부정, 중립을 3개의 새로운 컬럼으로 분리하여 저장
total_df[['제목_긍정점수', '제목_부정점수', '제목_중립점수']] = pd.DataFrame(title_sentiments.tolist(), index=total_df.index)
print(" -> 제목 감성 분석 완료!")


# 본문 감성 분석
print("\n'본문_정제' 컬럼에 대한 감성 분석을 시작합니다... (모든 점수 추출)")
body_sentiments = total_df['본문_정제'].progress_apply(analyze_all_sentiments)

# 긍정, 부정, 중립을 3개의 새로운 컬럼으로 분리하여 저장
total_df[['본문_긍정점수', '본문_부정점수', '본문_중립점수']] = pd.DataFrame(body_sentiments.tolist(), index=total_df.index)
print(" -> 본문 감성 분석 완료!")


# --------------------------------------------------------------------------
# 3-4. 최종 결과 확인 및 저장
# --------------------------------------------------------------------------
print("\n--- [ 감성 분석 최종 결과 샘플 ] ---")
# 새로 생성된 6개의 감성 점수 컬럼을 포함하여 결과 확인
display(total_df[['제목_정제', '제목_긍정점수', '제목_부정점수', '제목_중립점수',
                   '본문_정제', '본문_긍정점수', '본문_부정점수', '본문_중립점수']].head())

# 감성 분석이 완료된 최종 데이터프레임을 새로운 파일로 저장
final_output_path = "/content/drive/MyDrive/Colab Notebooks/final_blog_with_sentiment.pkl"
total_df.to_pickle(final_output_path)
print(f"\n최종 감성 분석 결과를 '{final_output_path}' 파일로 저장했습니다.")

In [ ]:
# 4. 일자별 제목, 본문의 감성 기사 수 집계 그래프
# --------------------------------------------------------------------------
# 1. 데이터 불러오기 및 준비
# --------------------------------------------------------------------------
try:
    df = pd.read_pickle("/content/drive/MyDrive/Colab Notebooks/카톡 업뎃(25.09.23) 반응 분석/final_blog_with_sentiment.pkl") # ★ 본인 파일 경로로 수정 ★

    # [수정] 날짜 형식으로 변환 (기존 코드 재확인)
    df['작성일_datetime'] = pd.to_datetime(df['작성일_정제'], errors='coerce')

    # [핵심 수정] 분석할 기간(2025년 9월~11월)으로 데이터 필터링
    start_date = '2025-09-11'
    end_date = '2025-11-30'
    df_filtered = df[(df['작성일_datetime'] >= start_date) & (df['작성일_datetime'] <= end_date)].copy()

    print(f"\n원본 데이터 {len(df)}건 중, 분석 기간({start_date}~{end_date})에 해당하는 데이터 {len(df_filtered)}건을 필터링했습니다.")

    df_filtered = df_filtered.set_index('작성일_datetime')

    # 최종 감성 컬럼 생성
    df_filtered['제목_최종감성'] = df_filtered.apply(lambda row: '긍정' if row['제목_긍정점수'] > row['제목_부정점수'] else '부정', axis=1)
    df_filtered['본문_최종감성'] = df_filtered.apply(lambda row: '긍정' if row['본문_긍정점수'] > row['본문_부정점수'] else '부정', axis=1)

except FileNotFoundError:
    print("오류: 데이터 파일을 찾을 수 없습니다.")
    df_filtered = None

if df_filtered is not None:
    # --------------------------------------------------------------------------
    # 2. 날짜별 긍정/부정 기사 수 집계 (필터링된 데이터 사용)
    # --------------------------------------------------------------------------
    title_sentiment_daily = df_filtered.groupby([df_filtered.index.date, '제목_최종감성']).size().unstack(fill_value=0)
    body_sentiment_daily = df_filtered.groupby([df_filtered.index.date, '본문_최종감성']).size().unstack(fill_value=0)

    # --------------------------------------------------------------------------
    # 3. 시각화 (폰트 속성 추가)
    # --------------------------------------------------------------------------
    plt.figure(figsize=(20, 12))

    # --- 제목 기준 그래프 ---
    plt.subplot(2, 1, 1)
    title_sentiment_daily.plot(kind='bar', stacked=False, ax=plt.gca(), color=['#3498db', '#e74c3c'])
    # [수정] fontproperties=font_prop 추가
    plt.title('날짜별 블로그 "제목"의 긍정/부정 기사 수', fontsize=16,  pad=20)
    plt.xlabel('날짜', fontsize=12)
    plt.ylabel('기사 수 (count)', fontsize=12)
    plt.xticks(rotation=45, ha='right') # ha='right'로 x축 라벨 정렬 개선
    plt.legend(title='감성')
    plt.grid(axis='y', linestyle='--', alpha=0.7)

    # --- 본문 기준 그래프 ---
    plt.subplot(2, 1, 2)
    body_sentiment_daily.plot(kind='bar', stacked=False, ax=plt.gca(), color=['#2ecc71', '#c0392b'])
    # [수정] fontproperties=font_prop 추가
    plt.title('날짜별 블로그 "본문"의 긍정/부정 기사 수', fontsize=16, pad=20)
    plt.xlabel('날짜', fontsize=12)
    plt.ylabel('기사 수 (count)', fontsize=12)
    plt.xticks(rotation=45, ha='right')
    plt.legend(title='감성')
    plt.grid(axis='y', linestyle='--', alpha=0.7)

    plt.tight_layout(pad=3.0)
    plt.show()